In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [53]:
df = pd.read_csv('/content/drive/MyDrive/Toxic_Classifier/train.csv')
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [5]:
df.shape

(159571, 8)

In [25]:
# https://stackoverflow.com/questions/43018030/replace-apostrophe-short-words-in-python
APPOS = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "i would",
"i'd" : "i had",
"i'll" : "i will",
"i'm" : "i am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "i have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}


In [9]:
import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
STOPWORDS = set(stopwords.words("english"))

lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer()
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [15]:
# func check_istoxic and get_istoxic
def check_istoxic(count):
  if count < 1:
        return 0
  else:
        return 1    
def get_istoxic(df):
    df['sum_of_toxic'] = df['toxic'] + df['severe_toxic'] + df['obscene'] + df['threat'] + df['insult'] + df['identity_hate']
    df['is_toxic'] = df['sum_of_toxic'].apply(lambda x: check_istoxic(x))
    df = df.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'sum_of_toxic'], axis=1)
    return df

In [16]:
def get_features_count(df):
    df['count_of_sent'] = df['comment_text'].apply(lambda x: len(re.findall('\n', str(x)))+1)
    df['count_of_word'] = df['comment_text'].apply(lambda x: len(str(x).split()))
    df['count_of_unique_word'] = df['comment_text'].apply(lambda x: len(set(str(x).split())))
    df['count_of_punctuations'] = df['comment_text'].apply(lambda x: len([s for s in str(x) if s in string.punctuation]))
    df['count_of_upper_words'] = df['comment_text'].apply(lambda x: len([s for s in str(x).split() if s.isupper()]))
    df['count_of_stopwords'] = df['comment_text'].apply(lambda x: len([s for s in str(x).lower().split() if s in STOPWORDS]))

    df['unique_word_percent'] = df['count_of_unique_word'] * 100 / df['count_of_word']
    df['punct_percent'] = df['count_of_punctuations'] * 100 / df['count_of_word']

    return df

In [17]:
def comment_processed(comment):
    comment = comment.lower()
    comment = re.sub('\\n', " ", comment)
    comment = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","",comment)
    comment = re.sub("\[\[.*\]","",comment)
    words = tokenizer.tokenize(comment)
    
    words = [APPOS[word] if word in APPOS else word for word in words]
    sent = " ".join(words)
    words = tokenizer.tokenize(sent)
    # stemming
    words = [lemmatizer.lemmatize(word, "v") for word in words]
    words = [w for w in words if not w in STOPWORDS]
    
    
    clean_sent = " ".join(words)
    return(clean_sent)

In [18]:
def get_comment_processed(df):
    df['comment_text'] = df['comment_text'].apply(lambda x: comment_processed(x))
    
    return df

In [19]:
def vader_polarity(sentence, threshold=0.1):
    scores = analyzer.polarity_scores(sentence)
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

In [20]:
def get_sent_scores(df):
    df['sent_scores'] = df['comment_text'].apply(lambda x: vader_polarity(x))
    
    return df

In [55]:
def preprocessing(df,drop_text=False):
    get_istoxic(df)
    get_features_count(df)
    get_comment_processed(df)
    # get_sent_scores(df)
    df = df.drop(['id'], axis=1)
    
    return df

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
# df
df_raw = preprocessing(df)
#df = preprocessing(df,drop_text=True)
# df
df_raw

In [58]:
y_labels = df_raw['is_toxic']
X_features = df_raw['comment_text']

X_features

0         explanation edit make username hardcore metall...
1         d'aww ! match background colour seemingly stic...
2         hey man , really try edit war . guy constantly...
3         " cannot make real suggestions improvement - w...
4                     , sir , hero . chance remember page ?
                                ...                        
159566    " :: : second time ask , view completely contr...
159567               ashamed horrible thing put talk page .
159568    spitzer umm , theres actual article prostituti...
159569    look like actually put speedy first version de...
159570    " ... really think understand . come idea bad ...
Name: comment_text, Length: 159571, dtype: object

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

tfvect = TfidfVectorizer(max_df=0.7 , stop_words="english")

tfid_x_train = tfvect.fit_transform(X_train)
tfid_x_test = tfvect.transform(X_test)
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)




In [68]:
y_pred = classifier.predict(tfid_x_test)
print(f'y_test:{y_test}')
print(f'y_pred:{y_pred}')


from sklearn.metrics import accuracy_score, confusion_matrix
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

y_test:74251     0
131406    1
120969    1
121827    0
4771      0
         ..
34111     0
16106     0
64414     0
87691     0
20951     0
Name: is_toxic, Length: 31915, dtype: int64
y_pred:[0 0 1 ... 0 0 0]
Accuracy: 95.02%


In [ ]:
import pickle
# import joblib
# pickle.dump(gridcv, open('/content/drive/MyDrive/Toxic_Classifier/model.pkl', 'wb'))
 # joblib.dump(gridcv, '/content/drive/MyDrive/Toxic_Classifier/model2.pkl')

In [63]:
/content/drive/MyDrive/Toxic_pickle.dump(classifier, open('Classifier/my-model.pkl', 'wb'))


In [64]:
pickle.dump(tfvect, open('/content/drive/MyDrive/Toxic_Classifier/tfidf.pkl', 'wb'))

In [70]:
from joblib import dump
dump(classifier, '/content/drive/MyDrive/Toxic_Classifier/my-model.joblib')

['/content/drive/MyDrive/Toxic_Classifier/my-model.joblib']

In [71]:
from sklearn.neural_network import MLPClassifier
mlp_classifier = MLPClassifier(
    hidden_layer_sizes=(50),
    activation='relu',
    solver='lbfgs',
    random_state=0,
    max_iter=10000
)

In [72]:
mlp_classifier.fit(tfid_x_train,y_train)
print("> Train completed")
y_pred = mlp_classifier.predict(tfid_x_test)
print(f'y_test:{y_test}')
print(f'y_pred:{y_pred}')

score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

> Train completed
y_test:74251     0
131406    1
120969    1
121827    0
4771      0
         ..
34111     0
16106     0
64414     0
87691     0
20951     0
Name: is_toxic, Length: 31915, dtype: int64
y_pred:[0 0 1 ... 0 0 0]
Accuracy: 92.73%


In [73]:
dump(mlp_classifier, '/content/drive/MyDrive/Toxic_Classifier/mlp-model.joblib')

['/content/drive/MyDrive/Toxic_Classifier/mlp-model.joblib']